# Database Function Test #1

## Objective

To determine if a Pandas DataFrame can be sent to a PostgreSQL function, using the `to_sql` method

## Steps

1. Create a table
2. Create a PostgreSQL function that INSERTs into the table
3. Create a DataFrame with data to be added to the table
4. Use the `to_sql` function, specifying the function name, where a table would normally be used

### Preparation

#### Imports

In [2]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine, text

#### Connect to Database

In [4]:
db_url = "postgresql+psycopg://localhost/patrick"
db_schema = "btc_rates"

engine = create_engine(db_url, echo=True)

@sa.event.listens_for(engine, "connect", insert=True)
def set_search_path(dbapi_connection, connection_record):
    """
    Set schema search path in database
    """
    sql = f"SET SESSION search_path TO {db_schema},public;"
    existing_autocommit = dbapi_connection.autocommit
    dbapi_connection.autocommit = True
    cursor = dbapi_connection.cursor()
    cursor.execute(sql)
    cursor.close()
    dbapi_connection.autocommit = existing_autocommit

### Step One: Create Table

In [12]:
tbl = "dt_test_1"
sqls = [
    f"DROP TABLE IF EXISTS {tbl};",
    f"CREATE TABLE {tbl}(id INT PRIMARY KEY, dt TIMESTAMPTZ, msg TEXT);",
]
sql = "\n".join(sqls)

with engine.connect() as conn:
    result = conn.execute(text(sql))
    conn.commit()

2024-12-07 09:47:35,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 09:47:35,225 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS dt_test_1;
CREATE TABLE dt_test_1(id INT PRIMARY KEY, dt TIMESTAMPTZ, msg TEXT);
2024-12-07 09:47:35,227 INFO sqlalchemy.engine.Engine [cached since 244.4s ago] {}
2024-12-07 09:47:35,235 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
result.rowcount

-1

## Conclusion

I didn't need to complete this experiment to verify that INSERT only works with tables (not functions)